In [ ]:
import sys
sys.path.append('..')
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
from src.modified_diffusers import StableDiffusionControlNetPipeline, ControlNetModel
from diffusers import AutoencoderKL, StableDiffusionPipeline
import torch
import numpy as np


In [ ]:
torch.cuda.get_device_name(0)

In [ ]:
ckpt_path='/mnt/d/repos/stable-diffusion-webui/models/Stable-diffusion/realdosmix_.safetensors'
pipe = StableDiffusionPipeline.from_ckpt(ckpt_path)
pipe.vae = AutoencoderKL.from_pretrained('lint/anime_vae')

#pipe = StableDiffusionPipeline.from_pretrained('lint/simpathizer')


In [ ]:
controlnet_path = '/mnt/d/repos/style_controlnet/models/realdosmix__animestyler2/checkpoint-10000'

pipe = StableDiffusionControlNetPipeline(
    **pipe.components,
    controlnet = ControlNetModel.from_pretrained(controlnet_path)
)



In [ ]:
pipe.controlnet = ControlNetModel.from_pretrained("../models/realdosmix__animestyler2/checkpoint-15401")

In [ ]:
pipe.feature_extractor = None
pipe.safety_checker = None
pipe.requires_safety_checker = False
#pipe.enable_xformers_memory_efficient_attention()

for component in pipe.components.values():
    if isinstance(component, torch.nn.Module):
        component.to('cuda', torch.float16)


In [ ]:
#prompt from https://civitai.com/gallery/76860, https://civitai.com/gallery/76862
prompt = "best quality, masterpiece,Dark hair, dark eyes, upper body, sun flare, outdoors, mountain, valley, sky. clouds, smiling"
negative_prompt = "(low quality, worst quality:1.4)"
controlnet_prompt = prompt


In [ ]:
from diffusers.schedulers import EulerAncestralDiscreteScheduler
pipe.scheduler = EulerAncestralDiscreteScheduler.from_pretrained('runwayml/stable-diffusion-v1-5', subfolder='scheduler')

In [ ]:
conditioning_scales = np.linspace(0, 1, 51)
#conditioning_scales = [0.0, 0.5, 1.0]
output_dir = '../archive/mountains'
os.makedirs(output_dir, exist_ok=True)

for cond_scale in conditioning_scales:
    result = pipe(
        prompt=prompt, 
        generator=torch.Generator().manual_seed(3283666171), 
        controlnet_conditioning_scale=cond_scale,
        guidance_scale=11,
        num_inference_steps=20,
        negative_prompt=negative_prompt,
        controlnet_prompt=controlnet_prompt,
        height=768,
    )

    result.images[0].save(f'{output_dir}/conditioning_scale_{np.around(cond_scale,2)}.png')

In [ ]:
# make gif

from glob import glob
from PIL import Image

output_dir = '../archive/brown_eyes'
save_dir = '../archive'
resolution = (256, 256)

frames = [Image.open(image).resize(resolution) for image in sorted(glob(f"{output_dir}/*.png"))]

frames[0].save(f"{save_dir}/brown_eyes.gif", format="GIF", append_images=frames+frames[::-1],
               save_all=True, duration=int(1000/15), loop=0)

In [ ]:
# make image grid

from PIL import Image
from torchvision.utils import make_grid
from torchvision.io import read_image
from pathlib import Path

images = []
image_dir = output_dir

for image_path in sorted(Path(image_dir).glob('*.?.png')):
    images.append(read_image(str(image_path)))

grid = make_grid(images, nrow=11)
grid = grid.permute(1,2,0).numpy()
image_grid = Image.fromarray(grid)
image_grid.save(Path(image_dir).parent/f'{Path(output_dir).name}_grid.png')